# Reorganize farmland (最適化案作成)

In [76]:
import geopandas as gpd
from pyproj import Geod
from shapely.geometry import Point, Polygon
from shapely.ops import unary_union

# Load the GeoJSON file
file_path = '../notebooks/data/geojson_filtered_by_settlement/筑地.geojson'
gdf = gpd.read_file(file_path)

gdf.head()

,polygon_uuid,land_type,issue_year,edit_year,history,last_polygon_uuid,prev_last_polygon_uuid,local_government_cd,point_lng,point_lat,...,DaichoId,daicho_shubetsu_cd,CommencementDate,EndStagesDate,UseIntentionInvestigationDate,Settlement_id,Settlement_name,Settlement_pref,Settlement_city,geometry
0,edad72c2-a9d9-44c5-85a7-b8a7b141eb1a,100,2024,2020,"[ { ""筆ポリゴンID"": ""4adb940c-36c9-40a7-bb7b-88a212...",4adb940c-36c9-40a7-bb7b-88a212840e2e,6e278408-d2b0-4d1e-845f-883dea2c08a1,082015,140.378240,36.375149,...,81009535,KNO01,None,None,None,006,筑地,茨城県,水戸市,"POLYGON ((140.3778 36.37503, 140.37778 36.3751..."
1,bbce7562-d4af-4c32-9256-ac23e0562af1,100,2024,2020,"[ { ""筆ポリゴンID"": ""6f3c12a9-7c9b-45e1-bf1a-211531...",6f3c12a9-7c9b-45e1-bf1a-211531f38051,681d5417-51c1-4b49-a071-b07b98622342,082015,140.377036,36.376148,...,"81009529, 81009541",KNO01,None,None,None,006,筑地,茨城県,水戸市,"POLYGON ((140.3766 36.37601, 140.37657 36.3761..."
2,e5d53da1-3199-4cea-b472-57a468504605,100,2024,2020,"[ { ""筆ポリゴンID"": ""308a48e7-fd6e-4f51-a35d-47afb4...",308a48e7-fd6e-4f51-a35d-47afb42a12fb,f9963c97-04f0-4e5b-8217-9605cb7a6b41,082015,140.377079,36.375928,...,81009530,KNO01,20210319,20260331,None,006,筑地,茨城県,水戸市,"POLYGON ((140.37664 36.37573, 140.37659 36.375..."
3,8af11e07-a310-40d4-bc6b-017f93b53afc,200,2024,2020,"[ { ""筆ポリゴンID"": ""586de863-0e1a-4d1c-a087-dfc007...",586de863-0e1a-4d1c-a087-dfc0072ab084,aa3501c6-ddc0-40a6-9de8-4bf951c50b85,082015,140.373818,36.369991,...,81009010,KNO01,None,None,None,006,筑地,茨城県,水戸市,"POLYGON ((140.37394 36.3701, 140.37386 36.3698..."
4,73a589a8-ed63-43d7-baf4-b315b386a308,100,2024,2020,"[ { ""筆ポリゴンID"": ""79b30ad5-940d-47ea-bfd6-a61271...",79b30ad5-940d-47ea-bfd6-a61271ca70ba,74bda683-7056-4952-b680-0c955f773a4f,082015,140.371795,36.371479,...,81009277,KNO01,20120319,20220331,None,006,筑地,茨城県,水戸市,"POLYGON ((140.37199 36.37171, 140.37214 36.371..."


In [8]:
gdf[:5]["point_lat"]

0    36.375149
1    36.376148
2    36.375928
3    36.369991
4    36.371479
Name: point_lat, dtype: float64

In [13]:
gdf.iloc[0]['geometry'].exterior.coords.xy

(array('d', [140.37779893, 140.37777543299998, 140.37866096499997, 140.378686893, 140.37779893]),
 array('d', [36.37502998799999, 36.375142756, 36.375272253, 36.375144836, 36.37502998799999]))

In [5]:
print(gdf.keys())

Index(['polygon_uuid', 'land_type', 'issue_year', 'edit_year', 'history',
       'last_polygon_uuid', 'prev_last_polygon_uuid', 'local_government_cd',
       'point_lng', 'point_lat', 'num_farm_pins', 'FarmCommitteeCd',
       'FarmCommitteeName', 'TodofukenCode', 'ShikuchosonCode', 'OazaCode',
       'ClassificationOfLandCodeName', 'ClassificationOfLand',
       'AreaOnRegistry', 'SectionOfNoushinhouCodeName', 'SectionOfNoushinhou',
       'SectionOfToshikeikakuhouCodeName', 'SectionOfToshikeikakuhou',
       'OwnerFarmIntentionCodeName', 'OwnerFarmIntention',
       'FarmerIndicationNumberHash', 'KindOfRightCodeName', 'KindOfRight',
       'RightSettingContentsCodeName', 'RightSettingContents',
       'UsageSituationInvestigationDate',
       'UsageSituationInvestigationResultCodeName',
       'UsageSituationInvestigationResult',
       'OwnerStatementIntentSurveyResultsCodeName',
       'OwnerStatementIntentSurveyResults',
       'UseIntentionAscertainmentResultCodeName',
       'Us

In [ ]:
class Farmer:
    def __init__(self, ID):
        self.ID = ID
        self.farms = gdf[gdf['FarmerIndicationNumberHash'] == ID]
        self.farm_centroid = self.calculate_centroid()
        self.total_farm_area = self.calculate_total_farm_area()
        self.basepoint1 = self.calculate_basepoint1()

    def calculate_centroid(self):
        # 緯度の平均
        lat = self.farms["point_lat"].mean()
        # 経度の平均
        lng = self.farms["point_lng"].mean()
        return (lng, lat)

    def calculate_total_farm_area(self):
        """ポリゴンの面積を計算する（球面積分を使用）"""
        geod = Geod(ellps="WGS84")
        area = 0
        for i in range(len(self.farms)):
            lon, lat = self.farms.iloc[i]["geometry"].exterior.coords.xy
            area += abs(geod.polygon_area_perimeter(lon, lat)[0]) # 単位は平方メートル
        return area

    def calculate_basepoint1(self):
        """ 一つ一つの農地の座標から1km範囲での自分の農地の面積が広い農地をその農家の中心農地として座標設定する。"""
        max_area = 0
        central_farm = None

        # 一つ一つの農地に対して、1km範囲での自分の農地の面積が広い農地を中心農地として設定する
        for farm in self.farms.geometry:
            # 1kmのバッファを作成
            buffer = farm.buffer(1/440)
            # バッファと交差する農地を取得
            intersecting_farms = self.farms[self.farms.geometry.intersects(buffer)]
            print(len(intersecting_farms))
            total_area = intersecting_farms.to_crs(epsg=32654).geometry.area.sum()
            print('Farmer ID:', self.ID, 'Total Area:', total_area)

            if total_area > max_area:
                max_area = total_area
                central_farm = farm

            print(max_area)

        return central_farm.centroid.coords[0]


In [125]:
# FarmerIndicationNumberHashのユニークな値を取得
farmerid_list = gdf['FarmerIndicationNumberHash'].unique()
print(farmerid_list)

['a504f94f9a34e120b11c26dcc561cc93'
 '0f1cd9d3f5b6f65893e4f31c96c52045, d0dbbea72d1f340a64f82d1060dc9a42'
 '745fb7229d1395c231177495b777fd2e' '2e20b8f76a7381f7e923f5f174579807'
 'a96dd9b2cfd69e2d52d31a0fee58d768' '430272ee3404f2f0f84462d72c5c84e9'
 'b678fcead885c5626ba43f2fe51398c5'
 '7db8af145bda49552f855ba395906a2f, 9171a59c2fab87fa3f28f839fb1db372, 8804b26aea1e028b08b9554aa81b504e, 430272ee3404f2f0f84462d72c5c84e9, 9d966c0b386ce1d8da0275c4f37e60b2, 1eec5a4995f5c1dd711f067b0cb0c4cd'
 '44b4a2f1d7f1089df78744f394945b34' '9eda0c58317630de6e3fbcd5a2ee3918'
 'd11c6aa5179c36432057bb2c395f7ac5' '4dcb84569dd9bf3a0911e8eb585078b2'
 '384ebc2b1149d39c31e8f6d020088847' '3d3db50f0f8bdc020dd68dd0dc6a61c0'
 'fda7674bced0a1d73ccd925ef01b5401'
 '5b195ed949578e29f05e761a3bfe86b9, a3d2f8858e3ea5cdca9414d3d7fb0864'
 '09abb0d6c5bdd2647c39648ee62b1418' '8804b26aea1e028b08b9554aa81b504e'
 '9b5518b423a09c210633bf87302b2688' '7d2394ffefeb91b209020cb735a24ce3'
 '2dacba93d45b0f46a25b29b985bd90e2'
 '2dacba93d45

In [162]:
farmer1 = Farmer("2dacba93d45b0f46a25b29b985bd90e2")


3
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 3881.7989955903804
3881.7989955903804
7
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 10279.290303669048
10279.290303669048
6
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 12117.51406655079
12117.51406655079
7
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 13175.110514021113
13175.110514021113
7
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 13175.110514021113
13175.110514021113
6
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 8068.645253907449
13175.110514021113
6
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 10720.72116754156
13175.110514021113
7
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 10279.290303669048
13175.110514021113
7
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 11906.14549072555
13175.110514021113
8
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2 Total Area: 11856.747891773195
13175.110514021113
8
Farmer ID: 2dacba93d45b0f46a25b29b985bd90e2

In [163]:
for farmerid in farmerid_list:
    farmer = Farmer(farmerid)
    print(farmer.ID, farmer.farm_centroid, farmer.total_farm_area, farmer.basepoint1)

1
Farmer ID: a504f94f9a34e120b11c26dcc561cc93 Total Area: 1089.6663141952263
1089.6663141952263
1
Farmer ID: a504f94f9a34e120b11c26dcc561cc93 Total Area: 769.8923396567313
1089.6663141952263
a504f94f9a34e120b11c26dcc561cc93 (np.float64(140.3782323745), np.float64(36.3733019175)) 1860.9045011848211 (140.3782395196929, 36.375148699919635)
1
Farmer ID: 0f1cd9d3f5b6f65893e4f31c96c52045, d0dbbea72d1f340a64f82d1060dc9a42 Total Area: 1501.4682974595044
1501.4682974595044
0f1cd9d3f5b6f65893e4f31c96c52045, d0dbbea72d1f340a64f82d1060dc9a42 (np.float64(140.377036427), np.float64(36.376148316)) 1502.5545369070023 (140.37703642744984, 36.37614831633448)
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935

In [164]:
# それぞれの農家の農地の重心をピンとしてGeojosnファイルに書き出す
import json
from shapely.geometry import Point


features = []

import json
from shapely.geometry import Point

features = []
for farmerid in farmerid_list:
    farmer = Farmer(farmerid)
    point = Point(farmer.basepoint1)
    print(farmer.farm_centroid)
    feature = {
        "type": "Feature",
        "properties": {
            "FarmerIndicationNumberHash": farmerid,
            "total_farm_area": farmer.total_farm_area
        },
        "geometry": {
            "type": "Point",
            "coordinates": [point.x, point.y]
        }
    }
    features.append(feature)

geojson = {
    "type": "FeatureCollection",
    "crs": {
        "type": "name",
        "properties": {
            "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
        }
    },
    "features": features
}

with open("farmers.geojson", "w") as f:
    json.dump(geojson, f)

1
Farmer ID: a504f94f9a34e120b11c26dcc561cc93 Total Area: 1089.6663141952263
1089.6663141952263
1
Farmer ID: a504f94f9a34e120b11c26dcc561cc93 Total Area: 769.8923396567313
1089.6663141952263
(np.float64(140.3782323745), np.float64(36.3733019175))
1
Farmer ID: 0f1cd9d3f5b6f65893e4f31c96c52045, d0dbbea72d1f340a64f82d1060dc9a42 Total Area: 1501.4682974595044
1501.4682974595044
(np.float64(140.377036427), np.float64(36.376148316))
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
1
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 389.4176691877219
10935.260418694847
(np.float64(140.37804438519998), np.float64(36.3743654134))
1
Farmer ID: 2e20b8f76a7381f7e

In [165]:
# farmの数が一番大きいfarmerを見つける
max_farm = 0
max_farmer = None
for farmerid in farmerid_list:
    farmer = Farmer(farmerid)
    if len(farmer.farms) > max_farm:
        max_farm = len(farmer.farms)
        max_farmer = farmer
print(max_farmer.ID, max_farm)

1
Farmer ID: a504f94f9a34e120b11c26dcc561cc93 Total Area: 1089.6663141952263
1089.6663141952263
1
Farmer ID: a504f94f9a34e120b11c26dcc561cc93 Total Area: 769.8923396567313
1089.6663141952263
1
Farmer ID: 0f1cd9d3f5b6f65893e4f31c96c52045, d0dbbea72d1f340a64f82d1060dc9a42 Total Area: 1501.4682974595044
1501.4682974595044
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
4
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 10935.260418694847
10935.260418694847
1
Farmer ID: 745fb7229d1395c231177495b777fd2e Total Area: 389.4176691877219
10935.260418694847
1
Farmer ID: 2e20b8f76a7381f7e923f5f174579807 Total Area: 335.16329753558
335.16329753558
5
Farmer ID: a96dd9b2cfd69e2d52d31a0fee58d768 Total Area: 9125.459411473737
9125.459411473737
3
Farmer ID: a96

In [166]:
# farmの数が一番大きいfarmerの農地を出力
max_farmer.farms.to_file("max_farmer.geojson", driver="GeoJSON")